In [1]:
!pip install nltk
!pip install torch
!pip install bert-score


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import nltk
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ismai\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ismai\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [3]:
import requests

API_ENDPOINT = "http://127.0.0.1:8080/translate"

def translate(target_lang:str, user_input:str) -> str:
    output = ""
    try:
        payload = {
            "targetlang": target_lang,
            "english_text": user_input
        }
        response = requests.post(API_ENDPOINT, json=payload, timeout=60)
        response_data = response.json()
        output=response_data["translated_text"]
    except Exception as e:
        print(e)
    return output

In [3]:
translate("Hindi","I am ismail")

'मैं इस्माइल हूँ।'

In [4]:
csv_file_path = "translated_output.csv"

In [ ]:
import pandas as pd
evaluation_df = pd.read_csv(csv_file_path)
evaluation_df

In [6]:
from nltk.translate.bleu_score import sentence_bleu
from nltk import word_tokenize

def evaluate_translation_bleu(translated_text:str, expected_text:str)->float:
    reference = word_tokenize(expected_text)
    candidate = word_tokenize(translated_text)
    score = sentence_bleu([reference], candidate)    
    return score

In [7]:
from nltk.translate import meteor_score
from nltk import word_tokenize

def evaluate_translation_meteor(translated_text:str, expected_text:str)->float:
    reference = word_tokenize(expected_text)
    candidate = word_tokenize(translated_text)
    score = meteor_score.meteor_score([reference], candidate)    
    return score

In [8]:
import bert_score

# limitation is 512 tokens at once, this can be solved by implimating chunking machanism 
def evaluate_translation_bert(translated_text:str, expected_text:str):
    reference = [expected_text]
    candidate = [translated_text]
    P, R, f1 = bert_score.score(candidate , reference, lang="others")
    return f1.numpy()[0]


/Users/sakina/Downloads/llm_translator/llm_base/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
#reference = "The quick brown fox jumps over the lazy dog"
#candidate = "The fast brown fox jumps over the lazy dog"


#reference = "I am Ismail"
#candidate = "I am Ismail"


reference = "[dataPoints]+ kritérium mentén"
candidate = "a [dataPoints]+ kritériumok alapján"

In [10]:
evaluate_translation_bleu(reference, candidate)

0.43012508513132625

In [11]:
evaluate_translation_meteor(reference, candidate)

0.5751811594202899

In [12]:
evaluate_translation_bert(reference, candidate)

0.89740247

In [13]:
target_lang = "Hungarian"
original_sentences = evaluation_df['english'].tolist()
expected_translations = evaluation_df['translated_value'].tolist()

In [14]:
translated_sentences = [translate(target_lang,sentence) for sentence in original_sentences]

In [ ]:
#evaluation_df['llm_generated_text']=translated_sentences
#evaluation_df.to_csv("evaluation.csv")

In [15]:
scores = []

for translated, expected in zip(translated_sentences, expected_translations):
    score = evaluate_translation_bert(translated, expected)
    scores.append(score)

average_score = sum(scores) / len(scores)


In [16]:
print(f"Average Score: {average_score:.4f}")


Average Score: 0.9264
